In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.models.mistral.modeling_mistral import *
from transformers.cache_utils import DynamicCache
import torch
from torch import nn
import copy
from types import MethodType
import pandas as pd
import datasets
from torch.utils.data import DataLoader
from itertools import islice
from tqdm import tqdm
import gc

In [13]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1").to('cuda')
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
Token = {v: k for k, v in tokenizer.get_vocab().items()}

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [2]:
dataset = datasets.load_from_disk(f'/home/wwu/msmarco_mistral_test').with_format('torch', device=torch.device('cuda'))
loader = DataLoader(dataset, batch_size=128)

In [4]:
torch.set_grad_enabled(False)

In [12]:
del model
gc.collect()
torch.cuda.empty_cache()

In [ ]:
ablate_model = copy.deepcopy(model)

In [8]:
params = dict(model.named_parameters())
for name, param in params.items():
    if 'mlp' in name:
        param.zero_()

In [10]:
num_batches = 20
it = islice(iter(loader), num_batches)
loss = 0
for batch in tqdm(it, total=num_batches):
    out = model(
        input_ids=batch['input_ids'], 
        labels=batch['input_ids'], 
        attention_mask=batch['attention_mask']
    )
    loss += out.loss.item()
loss /= num_batches

100%|██████████| 20/20 [02:10<00:00,  6.54s/it]


In [11]:
loss

10.92078218460083

full model: 2.243

mlp only: 6.938

attn only: 10.92

In [ ]:
batch = next(iter(loader))

In [ ]:
batch.keys()

In [ ]:
out = model(input_ids=batch['input_ids'], labels=batch['input_ids'], attention_mask=batch['attention_mask'])

In [ ]:
out.keys()

In [ ]:
out.loss